Load the core libraries for gridded data access (`xarray`), THREDDS catalog discovery (`siphon`), and vector wind diagnostics (`windspharm`).

In [ ]:
import xarray as xr
import numpy as np
from siphon.catalog import TDSCatalog
from windspharm.xarray import VectorWind

Connect to the UCAR THREDDS catalog and list the available 1° GFS “Best” datasets.

In [ ]:
cat_url = (
    "https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/"
    "Global_onedeg/catalog.xml?dataset=grib/NCEP/GFS/Global_onedeg/Best"
)

cat = TDSCatalog(cat_url)
list(cat.datasets)

Select the current “Best” dataset and open it via OPeNDAP. All metadata and variables are accessed lazily through xarray.

In [ ]:
best = next(iter(cat.datasets.values()))
dap_url = best.access_urls["OPENDAP"]
dap_url
ds = xr.open_dataset(dap_url, chunks={}, decode_times=True)
ds

Select the zonal and meridional wind components

In [ ]:
u_name = "u-component_of_wind_isobaric"
v_name = "v-component_of_wind_isobaric"

missing = [k for k in (u_name, v_name) if k not in ds.data_vars]
if missing:
    raise KeyError(f"Missing variables in dataset: {missing}")

Inspect the dimensions and coordinates of the zonal wind field to confirm expected axes (time, pressure, latitude, longitude).

In [ ]:
u0 = ds[u_name]
u0.dims, list(u0.coords)

Prepare the wind fields for analysis:
- Verify the presence of an isobaric pressure coordinate (in Pascals)
- Select the 200 hPa level
- Select the model time nearest to “now minus 6 hours”
- Normalize longitude to the range [-180, 180] if needed

The result is a single-time, single-level global wind field.

In [ ]:
u0 = ds[u_name]
v0 = ds[v_name]

pcoord = "isobaric"

if pcoord not in u0.coords:
    raise KeyError(f"Expected pressure coord '{pcoord}' not found. "
                   f"Available coords: {list(u0.coords)}")

assert u0[pcoord].attrs.get("units", "").lower() == "pa"
target_p = 20000.0

# Choose a time close to "now", offset by 6 hours (to better match available cycles)
now = np.datetime64("now", "s") - np.timedelta64(6, "h")
t_nearest = u0.sel(time1=now, method="nearest")["time1"].values

# Slice to nearest time and nearest pressure level
u = u0.sel(time1=t_nearest).sel({pcoord: target_p}, method="nearest")
v = v0.sel(time1=t_nearest).sel({pcoord: target_p}, method="nearest")

# Normalize longitude to [-180, 180] if dataset uses [0, 360]
lon_name = "lon" if "lon" in u.coords else next(c for c in u.coords if "lon" in c.lower())
lon_max = float(u[lon_name].max())
if lon_max > 180:
    new_lon = ((u[lon_name] + 180) % 360) - 180
    u = u.assign_coords({lon_name: new_lon}).sortby(lon_name)
    v = v.assign_coords({lon_name: new_lon}).sortby(lon_name)

print("now:", now)
print("selected time:", t_nearest)

u, v

Ensure latitude is ordered north-to-south, as required by windspharm. Fill missing values with zeros to satisfy windspharm’s assumptions.

In [ ]:
# Ensure latitude is north->south
lat_name = "lat" if "lat" in u.coords else [c for c in u.coords if "lat" in c.lower()][0]
if u[lat_name][0] < u[lat_name][-1]:
    u = u.sortby(lat_name, ascending=False)
    v = v.sortby(lat_name, ascending=False)

# Fill missing values (windspharm requires none)
u = u.fillna(0.0)
v = v.fillna(0.0)

float(u.isnull().sum()), float(v.isnull().sum())


Decompose the horizontal wind field into:
- Irrotational (divergent) components
- Nondivergent (rotational) components

Also compute the associated streamfunction and velocity potential.

In [ ]:
w = VectorWind(u, v)

uchi, vchi, upsi, vpsi = w.helmholtz()
sf, vp = w.sfvp()

uchi, sf


Assemble the original winds and derived fields into a single xarray Dataset. Attach minimal metadata and write the result to a NetCDF file for downstream analysis or visualization.

In [ ]:
out = xr.Dataset(
    data_vars=dict(
        u=u, v=v,
        uchi=uchi, vchi=vchi,
        upsi=upsi, vpsi=vpsi,
        sf=sf, vp=vp,
    ),
    coords=u.coords
)

# Helpful metadata
units_u = u.attrs.get("units", "m s-1")
units_v = v.attrs.get("units", "m s-1")

out["uchi"].attrs.update(long_name="Irrotational zonal wind", units=units_u)
out["vchi"].attrs.update(long_name="Irrotational meridional wind", units=units_v)
out["upsi"].attrs.update(long_name="Nondivergent zonal wind", units=units_u)
out["vpsi"].attrs.update(long_name="Nondivergent meridional wind", units=units_v)
out["sf"].attrs.update(long_name="Streamfunction")
out["vp"].attrs.update(long_name="Velocity potential")

path = "windspharm_products_gfs_onedeg_best.nc"
out.to_netcdf(path)
path


Quick plot sanity check.

In [ ]:
import matplotlib.pyplot as plt

out["sf"].plot()
plt.title("Streamfunction (sf) at ~200 hPa")
plt.show()